In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

# H0 anisotropy
## 1. With propagated error

In [12]:
def H0_dipole_variation_propagated(scan_bf, scan_bt, n, A_all, A_all_plus, A_all_minus):
    """
    Calculate the dipole variation with propagated error

    Arguments
    ---------
    scan_bf : pandas.DataFrame
        Best fit values of the scan.
    scan_bt : pandas.DataFrame
        Bootstrap values of the scan.
    n : float
        H0/H0_all = (A/A_all)^n.
    A_all : float
        Best fit value of A for all clusters.
    A_all_plus : float
        The positive 1-sigma uncertainty of A_all.
    A_all_minus : float
        The negative 1-sigma uncertainty of A_all. Here input as a positive number.
    """
    assert A_all_minus > 0
    assert A_all_plus > 0

    # Use the number of significance so they are automatically the dipole direction
    maxidx = scan_bf['n_sigma'].idxmax()
    minidx = scan_bf['n_sigma'].idxmin()

    A_all_sigma = np.max([A_all_plus, A_all_minus])

    # The + region
    Glon = scan_bf.loc[maxidx, 'Glon']
    Glat = scan_bf.loc[maxidx, 'Glat']

    A_max = scan_bf.loc[maxidx, 'A']
    A_max_distr = scan_bt[(scan_bt['Glon']==Glon) & (scan_bt['Glat']==Glat)]['A']
    A_max_distr = np.array(A_max_distr)

    # A_max_plus = np.percentile(A_max_distr, 84) - A_max
    # A_max_minus = A_max - np.percentile(A_max_distr, 16)
    # A_max_sigma = np.max([A_max_plus, A_max_minus])

    A_max_sigma = np.std(A_max_distr) # Use the standard deviation


    f_max = best_fit_H0_H0all_max = (A_max/A_all)**n
    H0_H0all_sigma_max = np.sqrt(n**2 * f_max**(2*n-2) / A_all**2 * A_max_sigma**2
                              + n**2 * f_max**(2*n-2) * A_max**2 / A_all**4 * A_all_sigma**2)
    
    # The - region
    Glon = scan_bf.loc[minidx, 'Glon']
    Glat = scan_bf.loc[minidx, 'Glat']

    A_min = scan_bf.loc[minidx, 'A']
    A_min_distr = scan_bt[(scan_bt['Glon']==Glon) & (scan_bt['Glat']==Glat)]['A']
    A_min_distr = np.array(A_min_distr)

    # A_min_plus = np.percentile(A_min_distr, 84) - A_min
    # A_min_minus = A_min - np.percentile(A_min_distr, 16)
    # A_min_sigma = np.max([A_min_plus, A_min_minus])

    A_min_sigma = np.std(A_min_distr) # Use the standard deviation

    f_min = best_fit_H0_H0all_min = (A_min/A_all)**n
    H0_H0all_sigma_min = np.sqrt(n**2 * f_min**(2*n-2) / A_all**2 * A_min_sigma**2
                              + n**2 * f_min**(2*n-2) * A_min**2 / A_all**4 * A_all_sigma**2)
    
    H0_H0all_sigma = np.sqrt(H0_H0all_sigma_max**2 + H0_H0all_sigma_min**2)
    return f_max-f_min, H0_H0all_sigma

In [13]:
import pandas as pd
import numpy as np
n = [1/2, 1/2, 2/5]
cone_size = [75, 60, 75]

for lc in range(2):
    df = pd.read_csv(f'../scripts/fit-all-lightcone{lc}.csv')
    for i, relation in enumerate(['LX-T', 'YSZ-T', 'M-T']):
        # extract A_all, A_all_lower, A_all_upper
        mask = df['relation'] == relation
        A_all = df[mask]['A'].values[0]
        A_all_lower = df[mask]['A_lower'].values[0]
        A_all_upper = df[mask]['A_upper'].values[0]
        
        scan_bf = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_best_fit_{relation}_θ{cone_size[i]}.csv')
        scan_bt = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_bootstrap_{relation}_θ{cone_size[i]}.csv')

        # Propagated error
        H0_variation, H0_variation_err = H0_dipole_variation_propagated(scan_bf, scan_bt, n[i], 
                A_all=A_all, A_all_plus=A_all_upper, A_all_minus=A_all_lower)

        print(relation, '&', f'{H0_variation:.3f}\\pm{H0_variation_err:.3f}')

LX-T & 0.045\pm0.025
YSZ-T & 0.091\pm0.031
M-T & 0.022\pm0.010
LX-T & 0.052\pm0.021
YSZ-T & 0.038\pm0.022
M-T & 0.014\pm0.010


## 2. Using number of significance directly, as Kostas suggested

In [20]:
def H0_dipole_variation_nosigma(scan_bf, n, A_all):
    """
    Calculate the dipole variation without propagated error

    Arguments
    ---------
    scan_bf : pandas.DataFrame
        Best fit values of the scan.
    scan_bt : pandas.DataFrame
        Bootstrap values of the scan.
    n : float
        H0/H0_all = (A/A_all)^n.
    A_all : float
        Best fit value of A for all clusters.
    """
    # Use the number of significance so they are automatically the dipole direction
    maxidx = scan_bf['n_sigma'].idxmax()
    # print(maxidx)
    minidx = scan_bf['n_sigma'].idxmin()
    # print(minidx)

    # Max number of significance to calculate the significance of H0 dipole variation
    maxnosigma = scan_bf.loc[maxidx, 'n_sigma']
    # print(maxnosigma)
    minnosigma = scan_bf.loc[minidx, 'n_sigma']
    assert maxnosigma + minnosigma == 0 # A sanity check, if they are truely the dipole direction

    # The + region
    max_Glon = scan_bf.loc[maxidx, 'Glon']
    max_Glat = scan_bf.loc[maxidx, 'Glat']

    A_max = scan_bf.loc[maxidx, 'A']

    f_max = (A_max/A_all)**n
    
    # The - region
    min_Glon = scan_bf.loc[minidx, 'Glon']
    min_Glat = scan_bf.loc[minidx, 'Glat']

    A_min = scan_bf.loc[minidx, 'A']

    f_min = (A_min/A_all)**n
 
    H0_variation = f_max-f_min
    H0_variation_err = H0_variation / maxnosigma

    return max_Glon, max_Glat, H0_variation, H0_variation_err, maxnosigma 


In [62]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../tools/')
import clusterfit as cf

n = [1/2, 1/2, 2/5]
cone_size = [75, 60, 75]
label = ['\\LXT', '\\YSZT', '\\MgasT']
H0_all = 68.1

for lc in range(2):
    df = pd.read_csv(f'../scripts/fit-all-lightcone{lc}.csv')
    print('\\midrule')
    print(f'\\multicolumn{{4}}{{c}}{{Lightcone {lc+1}}} \\\\')
    print('\\midrule')


    # Initialize the arrays. It works only with 90*90 resolution but what the fuck
    H0_var_sigma_maps = np.empty((3, 8100)) # Uncertainty of the DIPOLE variation in %, should be symmetrized by nature
    H0_var_maps       = np.empty((3, 8100)) # H0 variation maps in % 
    n_sigma_maps      = np.empty((3, 8100)) # Number of sigma of the dipole variation

    for i, relation in enumerate(['LX-T', 'YSZ-T', 'M-T']):
        # extract A_all, A_all_lower, A_all_upper
        mask = df['relation'] == relation
        A_all = df[mask]['A'].values[0]
        
        scan_bf = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_best_fit_{relation}_θ{cone_size[i]}.csv')
        # scan_bt = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_bootstrap_{relation}_θ{cone_size[i]}.csv')

        # Error using number of sigma
        lon, lat, H0_variation, H0_variation_err, max_no_sigma = H0_dipole_variation_nosigma(scan_bf, n[i], A_all=A_all)

        # In latex format for the table
        print(label[i], '&', f'$(\\ang{{{lon:.0f}}},\\ang{{{lat:.0f}}})$', '&', f'${H0_variation*100:.2f}\\pm{H0_variation_err*100:.2f}\\%$', '&', f'${max_no_sigma:.2f}\\sigma$ \\\\')

        # Calculate the H0 variation of the combined map
        best_fit_file = f'../data/fits/testrun/lightcone{lc}/scan_best_fit_{relation}_θ{cone_size[i]}.csv'
        best_fit = pd.read_csv(best_fit_file)

        # Load the A map
        A_map = best_fit['A'].values
        lons = best_fit['Glon'].values
        lats = best_fit['Glat'].values
        n_sigma_map = best_fit['n_sigma'].values

        # Replace the zeros with a small number
        n_sigma_map[n_sigma_map == 0] = 1e-5

        # Save to number of sigma maps
        n_sigma_maps[i] = n_sigma_map

        # Calculate the H0 map and symmetrize to obtain the dipole map
        H0_var_maps[i] = (A_map/A_all)**n[i] - 1
        H0_var_maps[i] = cf.make_dipole_map(H0_var_maps[i], lons, lats, central=0)
        # print(cf.find_dipole_in_dipole_map(H0_maps[i], lons, lats))
        
        # H0 error map. Both of the maps are symmetrized so H0_sigma_maps are also symmetrized
        H0_var_sigma_maps[i] = H0_var_maps[i] / n_sigma_maps[i]
        H0_var_sigma_maps[i][H0_var_sigma_maps[i] == 0] = 1e5




    # Combine the variation maps
    joint_H0_var_map = np.average(H0_var_maps, axis=0, weights=1/H0_var_sigma_maps**2)

    # Combine the number of significance (in a WRONG way for the thesis only)
    joint_sigma_map = np.sqrt(1/np.sum(1/H0_var_sigma_maps**2, axis=0))
    joint_n_sigma_map = joint_H0_var_map / joint_sigma_map # both of them are symmetrized, so joint_n_sigma_map is also symmetrized

    # Find the max dipole as highest significance
    max_n_sigma, min_n_sigma, lon, lat, maxloc = cf.find_dipole_in_dipole_map(joint_n_sigma_map, lons, lats)

    # Calculate the H0 variation of the combined map
    max_H0_var = joint_H0_var_map[maxloc] * 2 # The dipole is symmetrized, so we need to multiply by 2
    # joint_H0_map = joint_H0_map.reshape(90, 90).T # If you need to visualize

    max_var_err = max_H0_var / max_n_sigma

    print('Combined', '&', f'$(\\ang{{{lon:.0f}}},\\ang{{{lat:.0f}}})$', '&', f'${max_H0_var*100:.2f}\\pm{max_var_err*100:.2f}\\%$', '&', f'${max_n_sigma:.2f}\\sigma$ \\\\') 


\midrule
\multicolumn{4}{c}{Lightcone 1} \\
\midrule
\LXT & $(\ang{-144},\ang{8})$ & $4.47\pm2.05\%$ & $2.19\sigma$ \\
\YSZT & $(\ang{-68},\ang{20})$ & $9.07\pm2.45\%$ & $3.70\sigma$ \\
\MgasT & $(\ang{-84},\ang{12})$ & $2.22\pm0.72\%$ & $3.07\sigma$ \\
Combined & $(\ang{-68},\ang{20})$ & $3.15\pm0.78\%$ & $4.02\sigma$ \\
\midrule
\multicolumn{4}{c}{Lightcone 2} \\
\midrule
\LXT & $(\ang{128},\ang{-22})$ & $5.18\pm1.61\%$ & $3.21\sigma$ \\
\YSZT & $(\ang{128},\ang{-4})$ & $3.78\pm1.54\%$ & $2.46\sigma$ \\
\MgasT & $(\ang{124},\ang{-16})$ & $1.38\pm0.64\%$ & $2.17\sigma$ \\
Combined & $(\ang{128},\ang{-16})$ & $2.50\pm0.71\%$ & $3.55\sigma$ \\


In [61]:
import numpy as np
# Given values and uncertainties
values = np.array([2.4, 9.1, 2.5])
uncertainties = np.array([1.8, 2.4, 0.9])
# Calculate weights
weights = 1 / uncertainties**2
# Calculate the weighted mean
weighted_mean = np.sum(weights * values) / np.sum(weights)
# Calculate the uncertainty of the weighted mean
weighted_mean_uncertainty = np.sqrt(1 / np.sum(weights))
weighted_mean, weighted_mean_uncertainty
print("Weighted Mean:", weighted_mean)
print("Uncertainty of the Weighted Mean:", weighted_mean_uncertainty)

Weighted Mean: 3.1494382022471914
Uncertainty of the Weighted Mean: 0.7631984736045793


In [60]:
print(maxloc)
for i in range(len(H0_var_maps)):
    print((H0_var_maps[i,maxloc])*2)
    print((H0_var_sigma_maps[i,maxloc])*2)

2575
0.02425908626760498
0.018231384594117588
0.0906776782566856
0.024539030354470262
0.024867678056429532
0.009266344056124696


What if we combine only LX and YSZ? considering the correlation between relations

In [63]:

import pandas as pd
import numpy as np
import sys
sys.path.append('../tools/')
import clusterfit as cf

n = [1/2, 1/2, 2/5]
cone_size = [75, 60, 75]
label = ['\\LXT', '\\YSZT', '\\MgasT']
H0_all = 68.1

for lc in range(2):
    df = pd.read_csv(f'../scripts/fit-all-lightcone{lc}.csv')
    print('\\midrule')
    print(f'\\multicolumn{{4}}{{c}}{{Lightcone {lc+1}}} \\\\')
    print('\\midrule')


    # Initialize the arrays. It works only with 90*90 resolution but what the fuck
    H0_var_sigma_maps = np.empty((2, 8100)) # Uncertainty of the DIPOLE variation in %, should be symmetrized by nature
    H0_var_maps       = np.empty((2, 8100)) # H0 variation maps in % 
    n_sigma_maps      = np.empty((2, 8100)) # Number of sigma of the dipole variation

    for i, relation in enumerate(['LX-T', 'YSZ-T']):
        # extract A_all, A_all_lower, A_all_upper
        mask = df['relation'] == relation
        A_all = df[mask]['A'].values[0]
        
        scan_bf = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_best_fit_{relation}_θ{cone_size[i]}.csv')
        # scan_bt = pd.read_csv(f'../data/fits/testrun/lightcone{lc}/scan_bootstrap_{relation}_θ{cone_size[i]}.csv')

        # Error using number of sigma
        lon, lat, H0_variation, H0_variation_err, max_no_sigma = H0_dipole_variation_nosigma(scan_bf, n[i], A_all=A_all)

        # In latex format for the table
        print(label[i], '&', f'$(\\ang{{{lon:.0f}}},\\ang{{{lat:.0f}}})$', '&', f'${H0_variation*100:.2f}\\pm{H0_variation_err*100:.2f}\\%$', '&', f'${max_no_sigma:.2f}\\sigma$ \\\\')

        # Calculate the H0 variation of the combined map
        best_fit_file = f'../data/fits/testrun/lightcone{lc}/scan_best_fit_{relation}_θ{cone_size[i]}.csv'
        best_fit = pd.read_csv(best_fit_file)

        # Load the A map
        A_map = best_fit['A'].values
        lons = best_fit['Glon'].values
        lats = best_fit['Glat'].values
        n_sigma_map = best_fit['n_sigma'].values

        # Replace the zeros with a small number
        n_sigma_map[n_sigma_map == 0] = 1e-5

        # Save to number of sigma maps
        n_sigma_maps[i] = n_sigma_map

        # Calculate the H0 map and symmetrize to obtain the dipole map
        H0_var_maps[i] = (A_map/A_all)**n[i] - 1
        H0_var_maps[i] = cf.make_dipole_map(H0_var_maps[i], lons, lats, central=0)
        # print(cf.find_dipole_in_dipole_map(H0_maps[i], lons, lats))
        
        # H0 error map. Both of the maps are symmetrized so H0_sigma_maps are also symmetrized
        H0_var_sigma_maps[i] = H0_var_maps[i] / n_sigma_maps[i]
        H0_var_sigma_maps[i][H0_var_sigma_maps[i] == 0] = 1e5




    # Combine the variation maps
    joint_H0_var_map = np.average(H0_var_maps, axis=0, weights=1/H0_var_sigma_maps**2)

    # Combine the number of significance (in a WRONG way for the thesis only)
    joint_sigma_map = np.sqrt(1/np.sum(1/H0_var_sigma_maps**2, axis=0))
    joint_n_sigma_map = joint_H0_var_map / joint_sigma_map # both of them are symmetrized, so joint_n_sigma_map is also symmetrized

    # Find the max dipole as highest significance
    max_n_sigma, min_n_sigma, lon, lat, maxloc = cf.find_dipole_in_dipole_map(joint_n_sigma_map, lons, lats)

    # Calculate the H0 variation of the combined map
    max_H0_var = joint_H0_var_map[maxloc]*2
    # joint_H0_map = joint_H0_map.reshape(90, 90).T # If you need to visualize

    max_var_err = max_H0_var / max_n_sigma

    print('Combined', '&', f'$(\\ang{{{lon:.0f}}},\\ang{{{lat:.0f}}})$', '&', f'${max_H0_var*100:.2f}\\pm{max_var_err*100:.2f}\\%$', '&', f'${max_n_sigma:.2f}\\sigma$ \\\\') 


\midrule
\multicolumn{4}{c}{Lightcone 1} \\
\midrule
\LXT & $(\ang{-144},\ang{8})$ & $4.47\pm2.05\%$ & $2.19\sigma$ \\
\YSZT & $(\ang{-68},\ang{20})$ & $9.07\pm2.45\%$ & $3.70\sigma$ \\
Combined & $(\ang{-68},\ang{22})$ & $5.20\pm1.52\%$ & $3.42\sigma$ \\
\midrule
\multicolumn{4}{c}{Lightcone 2} \\
\midrule
\LXT & $(\ang{128},\ang{-22})$ & $5.18\pm1.61\%$ & $3.21\sigma$ \\
\YSZT & $(\ang{128},\ang{-4})$ & $3.78\pm1.54\%$ & $2.46\sigma$ \\
Combined & $(\ang{136},\ang{-16})$ & $4.45\pm1.21\%$ & $3.67\sigma$ \\
